# Create a sphere using PyVista

---


This notebook outlines different ways to generate a sphere using PyVista.


Import required modules.


In [ ]:
import pyvista as pv
import numpy as np
from scipy.spatial import ConvexHull
import numerical_geometry as ng

---

## 1 - Generate a sphere using PyVista's `pv.Sphere()` method


Generate and plot the sphere.


In [ ]:
sphere_mesh = pv.Sphere(radius=1, theta_resolution=20, phi_resolution=20)
sphere_mesh.plot(show_edges=True)

---

## 2 - Generating vertices


### 2.1 - Generate vertices in spherical coordinates


Generate arrays of $r$, $\theta$, $\phi$ values.


In [ ]:
r = np.array([1])
theta = np.linspace(0, np.pi, 20)
phi = np.linspace(0, 2 * np.pi, 40)

Convert from spherical coordinates to Cartesian coordinates. Combine the $x$, $y$, $z$ values into an array, `vertices_spherical`.


In [ ]:
r_v, theta_v, phi_v = np.meshgrid(np.array([1]), theta, phi)

x_v = r_v * np.sin(theta_v) * np.cos(phi_v)
y_v = r_v * np.sin(theta_v) * np.sin(phi_v)
z_v = r_v * np.cos(theta_v)

vertices_spherical = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

Create a PyVista mesh; this mesh only has vertices (i.e. it is a point cloud). To make visualizing the point cloud easier, we will perform a Delaunay 3D tessellation before plotting.


In [ ]:
sphere_mesh_spherical = pv.PolyData(vertices_spherical)

pl = pv.Plotter()
pl.add_mesh(sphere_mesh_spherical.delaunay_3d())
pl.add_mesh(
    sphere_mesh_spherical.points,
    color="black",
    point_size=5,
    render_points_as_spheres=True,
)
pl.show()

### 2.2 - Generate vertices in Cartesian coordinates


Generate arrays of $x$, $y$ values. Remove coordinate pairs which don't satisfy $x^2 + y^2 \leq 1$.


In [ ]:
x = np.linspace(-1, 1, 20)
y = np.linspace(-1, 1, 20)

x_v, y_v = np.meshgrid(x, y)
xy_coordinates = np.c_[x_v.reshape(-1), y_v.reshape(-1)]

radius_squared = xy_coordinates[:, 0] ** 2 + xy_coordinates[:, 1] ** 2
mask = radius_squared <= 1.0
xy_coordinates = xy_coordinates[mask]
radius_squared = radius_squared[mask]

Create an array, `vertices_cartesian`, to store the vertices. Populate it with the $(x, y)$ coordinates (twice).


In [ ]:
num_coords = xy_coordinates.shape[0]
vertices_cartesian = np.zeros((num_coords * 2, 3), dtype=float)

vertices_cartesian[0:num_coords, 0:2] = xy_coordinates
vertices_cartesian[num_coords : 2 * num_coords, 0:2] = xy_coordinates

Calculate the $z$ coordinates for each $(x, y)$ coordinate using the formula $z = \pm \sqrt{1 - x^2 - y^2}$. Remove duplicate vertices.


In [ ]:
vertices_cartesian[0:num_coords, 2] = np.sqrt(1 - radius_squared)
vertices_cartesian[num_coords : 2 * num_coords, 2] = (
    vertices_cartesian[0:num_coords, 2] * -1
)
vertices_cartesian = np.unique(vertices_cartesian, axis=0)

Create a PyVista mesh; this mesh only has vertices (i.e. it is a point cloud). To make visualizing the point cloud easier, we will perform a Delaunay 3D tessellation before plotting.


In [ ]:
sphere_mesh_cartesian = pv.PolyData(vertices_cartesian)

pl = pv.Plotter()
pl.add_mesh(sphere_mesh_cartesian.delaunay_3d())
pl.add_mesh(
    sphere_mesh_cartesian.points,
    color="black",
    point_size=5,
    render_points_as_spheres=True,
)
pl.show()

---

## 3 - Tessellating a 3D point cloud


The vertices generated in spherical coordinates in the previous section looked better than the vertices generated in Cartesian coordinates, so we will use the former method.


In [ ]:
r = np.array([1])
theta = np.linspace(0, np.pi, 20)
phi = np.linspace(0, 2 * np.pi, 40)

r_v, theta_v, phi_v = np.meshgrid(np.array([1]), theta, phi)

x_v = r_v * np.sin(theta_v) * np.cos(phi_v)
y_v = r_v * np.sin(theta_v) * np.sin(phi_v)
z_v = r_v * np.cos(theta_v)

vertices = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

sphere_mesh = pv.PolyData(vertices)

There are several algorithms we can use to tessellate the 3D point cloud; we will use PyVista's `delaunay_3d()` method and SciPy's `spatial.ConvexHull()` method.


Perform the tessellation using PyVista's `delaunay_3d()` method.


In [ ]:
sphere_mesh_delaunay = pv.PolyData(sphere_mesh.points).delaunay_3d()
sphere_mesh_delaunay.plot(show_edges=True)

Generate a triangles array using SciPy's `spatial.ConvexHull` method. Convert the triangles array to the format expected by PyVista, and create a new mesh.


In [ ]:
faces = ConvexHull(sphere_mesh.points).simplices
sphere_mesh_convex_hull = pv.PolyData(sphere_mesh.points)
sphere_mesh_convex_hull.faces = utils.numpy_faces_to_pyvista(faces)
sphere_mesh_convex_hull.plot(show_edges=True)

---

## 4 - Tessellating a 2D point cloud


The previous methods outlined in section 3 followed the following procedure:

1. Generate an array of $(\theta, \phi)$ values.
2. Convert to Cartesian coordinates.
3. Tessellate the 3D point cloud.

In this section, we will swap steps 2 and 3, so that the procedure reads as follows:

1. Generate an array of $(\theta, \phi)$ values.
2. Tessellate the 2D point cloud of $(\theta, \phi)$ values.
3. Convert to Cartesian coordinates, while keeping the same tessellation.

The code written in this section is implemented in the `utils.sphere()` function.


Define helper functions


In [ ]:
def add_pole_faces(mesh, num_azimuthal_angles):
    """
    Add pole faces
    ==============

    Appends all of the faces involving the poles to the mesh.
    This function assumes that `wrap_phi` has not been called.
    """
    # Extract the faces from the PyVista mesh.
    faces = utils.pyvista_faces_to_numpy(mesh.faces)
    max_vertex = np.max(faces)

    # Remove all faces involving the poles.
    bulk_faces = faces[
        ~((faces == 0).any(axis=1) | (faces == np.max(faces)).any(axis=1))
    ]

    # Generate faces made by the North pole.
    north_faces = np.zeros((num_azimuthal_angles - 1, 3), dtype=int)
    for i in range(num_azimuthal_angles - 1):
        north_faces[i, :] = np.array([i + 2, 0, i + 1])

    # Generate faces made by the South pole.
    south_faces = np.zeros((num_azimuthal_angles - 1, 3), dtype=int)
    for i in range(num_azimuthal_angles - 1):
        south_faces[i, :] = np.array(
            [
                max_vertex,
                max_vertex - num_azimuthal_angles + i + 1,
                max_vertex - num_azimuthal_angles + i,
            ]
        )

    return pv.PolyData(
        mesh.points,
        utils.numpy_faces_to_pyvista(np.vstack((north_faces, bulk_faces, south_faces))),
    )


def wrap_phi(mesh, num_polar_angles, num_azimuthal_angles):
    """
    Wrap phi
    ========

    This function connects the points with the largest phi values to the points phi = 0.
    This function assumes that `add_pole_faces` has already been called.
    """
    # Extract the faces from the PyVista mesh.
    faces = utils.pyvista_faces_to_numpy(mesh.faces)
    max_vertex = np.max(faces)

    # Wrap the phi coordinate in the bulk.
    wrapping_faces = np.zeros((2 * (num_polar_angles - 1) + 2, 3), dtype=int)
    for i in range(num_polar_angles - 1):
        # Make the first triangle.
        wrapping_faces[2 * i, :] = np.array(
            [
                (i + 2) * num_azimuthal_angles,
                (i * num_azimuthal_angles) + 1,
                (i + 1) * num_azimuthal_angles,
            ]
        )

        # Make the second triangle.
        wrapping_faces[(2 * i) + 1, :] = np.array(
            [
                (i + 2) * num_azimuthal_angles,
                (i + 1) * num_azimuthal_angles + 1,
                (i * num_azimuthal_angles) + 1,
            ]
        )

    # Wrap the phi coordinates at the poles.
    wrapping_faces[-2, :] = np.array([1, num_azimuthal_angles, 0])
    wrapping_faces[-1, :] = np.array(
        [max_vertex, max_vertex - 1, max_vertex - num_azimuthal_angles]
    )

    return pv.PolyData(
        mesh.points, utils.numpy_faces_to_pyvista(np.vstack((faces, wrapping_faces)))
    )


def correct_mesh(mesh, num_polar_angles, num_azimuthal_angles):
    """
    Correct mesh
    ============

    Corrects the tessellation produced by the `delaunay_2d()` function, and returns a new mesh.
    """
    mesh_1 = add_pole_faces(mesh, num_azimuthal_angles)
    mesh_2 = wrap_phi(mesh_1, num_polar_angles, num_azimuthal_angles)
    return mesh_2

### 4.1 - Make a 2D mesh


We will define the vertices in spherical, $(r, \theta, \phi)$, coordinates. For a sphere, $r=1$, so if we plot the mesh, it will look like a 2D plane. We can perform a tessellation of the mesh using the `delaunay_2d()` function; it should get the tessellation mostly correct, since the surface is 2D.


In [ ]:
# Generate arrays of theta, phi values for all points, excluding the poles.
N = 20
theta = np.linspace(0, np.pi, N)[1:-1]
phi = np.linspace(0, 2 * np.pi, 2 * N + 1)[:-1]

# Make an array of vertices for the bulk of the sphere.
r_v, theta_v, phi_v = np.meshgrid(np.array([1]), theta, phi)
sphere_vertices_spherical_no_poles = np.c_[
    r_v.reshape(-1), theta_v.reshape(-1), phi_v.reshape(-1)
]

# Add the poles to the array of vertices.
sphere_vertices_spherical = np.zeros(
    (sphere_vertices_spherical_no_poles.shape[0] + 2, 3), dtype=float
)
sphere_vertices_spherical[0, :] = [1.0, 0.0, 0.0]
sphere_vertices_spherical[1:-1, :] = sphere_vertices_spherical_no_poles
sphere_vertices_spherical[-1, :] = [1.0, np.pi, 0.0]

# Create a PyVista mesh, and perform and plot a Delaunay tessellation.
sphere_mesh_spherical_delaunay = pv.PolyData(sphere_vertices_spherical).delaunay_2d()
sphere_mesh_spherical_delaunay.plot(show_edges=True)

### 4.2 - Modify the faces


The Delaunay tessellation correctly tessellated some features of our mesh, but not others. We want the North pole ($\theta=0$) and the South pole ($\theta=\pi$) to be connnected to all of the adjacent $\theta$ values. This is handled by the `add_pole_faces()` function. We also need to connect the points with the largest $\phi$ values to the points at $\phi=0$. This is handled by the `wrap_phi()` function. Both of these functions are lumped into the `correct_mesh()` function, which should produce a correctly tessellated 2D mesh when called. The 2D mesh looks quite weird when plotted, as $\phi=0$ is equivalent to $\phi=2 \pi$, so there are faces connecting the largest $\phi$ edge to the $\phi = 0$ edge; this disappears when we plot the mesh in Cartesian coordinates.


In [ ]:
num_polar_angles = len(theta)
num_azimuthal_angles = len(phi)
sphere_mesh_spherical_custom = correct_mesh(
    sphere_mesh_spherical_delaunay, num_polar_angles, num_azimuthal_angles
)
sphere_mesh_spherical_custom.plot(show_edges=True)

### 4.3 - Map the 2D mesh to a 3D surface


Now that the 2D mesh is correctly tessellated, we can map our spherical, $(r, \theta, \phi)$ vertices to Cartesian, $(x, y, z)$, vertices, while keeping the same tessellation. This should result in a correctly tessellated sphere.


In [ ]:
# Make an array storing the vertices in Cartesian coordinates.
x_v = r_v * np.sin(theta_v) * np.cos(phi_v)
y_v = r_v * np.sin(theta_v) * np.sin(phi_v)
z_v = r_v * np.cos(theta_v)
sphere_vertices_cartesian_no_poles = np.c_[
    x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)
]

# Add the poles to the array of vertices.
sphere_vertices_cartesian = np.zeros(
    (sphere_vertices_cartesian_no_poles.shape[0] + 2, 3), dtype=float
)
sphere_vertices_cartesian[0, :] = [0.0, 0.0, 1.0]
sphere_vertices_cartesian[1:-1, :] = sphere_vertices_cartesian_no_poles
sphere_vertices_cartesian[-1, :] = [0.0, 00, -1.0]

# Create and plot a new mesh, using the triangles array generated for the 2D mesh.
sphere_mesh_cartesian_custom = pv.PolyData(
    sphere_vertices_cartesian, sphere_mesh_spherical_custom.faces
)
sphere_mesh_cartesian_custom.plot(show_edges=True, color="lightblue")